In [2]:
import numpy as np
import scipy.io
import torch

from VariGrad import VG_R, Encoder, Decoder
import matplotlib.pyplot as plt
from pylab import get_cmap

In [ ]:
def plot3Dgraph(ax,a,color="blue"):
    a=a.cpu()
    t=a[:,0:28]
    la=torch.cat([a[:,0:1],a[:,29:65]],dim=1)
    ra=torch.cat([a[:,0:1],a[:,66:102]],dim=1)
    ll=torch.cat([a[:,27:28],a[:,103:136]],dim=1)
    rl=torch.cat([a[:,27:28],a[:,137:170]],dim=1)
    h=torch.cat([a[:,0:1],a[:,171:]],dim=1)
    ax.plot3D(t[0], t[2], t[1],color=color)
    ax.plot3D(la[0], la[2], la[1],color=color)
    ax.plot3D(ra[0], ra[2], ra[1],color=color)
    ax.plot3D(ll[0], ll[2], ll[1],color=color)
    ax.plot3D(rl[0], rl[2], rl[1],color=color)
    ax.plot3D(h[0], h[2], h[1],color=color)

In [ ]:
batchsize = 10
trainsize=0
mat = scipy.io.loadmat("shape_graphs_testing.mat")
template=torch.from_numpy(mat["template"]).cuda().transpose(0,1)
edges=torch.from_numpy(mat['tedges']).long().cuda()
array=mat['graphs'].T
arraye=mat['edges'].T

In [ ]:
VGReconstuction = VG_R(template, edges ,trainsize, 100)

In [ ]:
trainingedges,trainingdata,testedges,testdata = VGReconstuction.load_data(array, arraye)

In [ ]:
VGReconstuction.load("trained_models/EncoderSG","trained_models/DecoderSG")

In [ ]:
loss=VGReconstuction.evaluate_loss(testdata,testedges,10,False)
print(loss.item())

In [ ]:
codes = VGReconstuction.E(testdata,testedges)
reconstructed =VGReconstuction.D(codes).detach()

In [ ]:
cm=get_cmap('hsv')
for i in range(67):
    print(i)
    a= reconstructed[i,:]+template[:]
    b=testdata[i].cpu()

    plt.figure(figsize=(10,10))    
    ax = plt.axes(projection='3d')
    plot3Dgraph(ax,a)    
    ax.scatter(b[0].cpu(),b[2].cpu(),b[1].cpu(), color=cm(np.linspace(0,1,testdata[i].shape[1])))
    plt.show()